[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **SAM3 Batch Inference**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

Perform batch inference on images using SAM3 (Segment Anything Model v3) with text prompts and export annotations in COCO-JSON format. Seamlessly upload results to Labellerr for review and refinement.

## **INSTALLATION**

Run the below cell and follow the instructions in the [README.md](README.md)

In [ ]:
# !git clone https://github.com/facebookresearch/sam3.git
# %pip install git+https://github.com/Labellerr/SDKPython.gitQ

## **Download SAM3 Model from Huggingface**


### SAM3 Batch Inference: Model Download Instructions

To perform SAM3 batch inference, you need to download the SAM3 model from the Hugging Face model hub.

**SAM 3 (Segment Anything Model v3)** is developed by Meta and currently requires access permission.

### Steps to get the SAM3 Model:

1.  **Step 1:** Visit the official [SAM 3 repository on Hugging Face](https://huggingface.co/facebook/sam3).
2.  **Step 2:** Request access (log in to Hugging Face and fill out the access form).
3.  **Step 3:** Once you’re approved, download the model checkpoint from files and versions (e.g., `sam3.pt`) in the designated model folder.

## **Import Required functions**

In [13]:
# This function is responsible for executing the batch inference process using the SAM3 model,
from sam3_batch_inference import run_batch_inference

# This function is designed to handle the batch uploading of pre-generated annotations to a Labellerr project.
from batch_upload_preannot import upload_preannotations

## **Fill the parameters for SAM3 batch inference**

In [14]:
# ============================================================
# CONFIGURATION - Fill this values
# ============================================================

# Path to the folder containing input images.
# Example: "flower_sample_img"
INPUT_FOLDER = "flower_sample_img"

# Path to the pre-trained model checkpoint file (.pt).
# This file contains the weights for the SAM3 model.
# Example: r"model\sam3.pt"
MODEL_CHECKPOINT = r"model\sam3.pt"

# A list of text prompts to be used for segmenting objects within the images.
# Each string in the list represents a different object or class to detect.
# Example: ["Red Flower", "Yellow Flower", "White Flower", "Violet Flower"]
TEXT_PROMPTS = ["Red Flower", 
                "Yellow Flower", 
                "White Flower",
                "Violet Flower",
                ]

In [15]:
# Call the batch inference function with specified parameters.
run_batch_inference(
    INPUT_FOLDER,       # Path to the input folder containing images for inference.
    TEXT_PROMPTS,       # List of text prompts to guide the segmentation model.
    MODEL_CHECKPOINT,   # Path to the pre-trained model checkpoint file (.pth).
)

Loading SAM3 model from: model\sam3.pt
Model loaded successfully
Found 7 images to process
Text prompts (4): ['Red Flower', 'Yellow Flower', 'White Flower', 'Violet Flower']
Confidence threshold: 0.4
Segmentation format: polygon
Processing strategy: Prompt-sequential (memory optimized)

Processing prompt 1/4: 'Red Flower'


[Red Flower]: 100%|██████████| 7/7 [00:27<00:00,  3.97s/it]



Processing prompt 2/4: 'Yellow Flower'


[Yellow Flower]: 100%|██████████| 7/7 [00:20<00:00,  2.90s/it]



Processing prompt 3/4: 'White Flower'


[White Flower]: 100%|██████████| 7/7 [00:28<00:00,  4.09s/it]



Processing prompt 4/4: 'Violet Flower'


[Violet Flower]: 100%|██████████| 7/7 [00:12<00:00,  1.71s/it]



Processing complete!
Total images processed: 7
Total categories: 4
Total annotations: 402
Average detections per image: 57.43

Per-category breakdown:
  - Red Flower: 131 detections
  - Yellow Flower: 102 detections
  - White Flower: 155 detections
  - Violet Flower: 14 detections

Output saved to: SAM3_Results/flower_sample_img/annotations.json


## **Review SAM3 annotations on Labellerr**

Here's how to set up your environment for reviewing SAM3 annotations:

1.  Go to Labellerr and create a dataset with your image folder.
2.  Create a project from that dataset. Copy the `project_id` and paste it into the `PROJECT_ID` variable.
3.  From the Labellerr API tab, copy the `API Key`, `API SECRET`, and `CLIENT ID`. Then, create a `.env` file and paste the values into it as follows:

    ```
    API_KEY = ""
    API_SECRET = ""
    CLIENT_ID = ""
    ```

    **NOTE:** Ensure the `.env` file is in the same directory as this notebook, and the variable names match the strings shown above exactly.
4.  Copy the path of the result annotations folder (where annotations are stored) and paste it into the `ANNOTATION_DIR` variable.

In [16]:
PROJECT_ID = "lynelle_additional_gibbon_85467"
ANNOTATION_DIR = r"SAM3_Results\flower_sample_img"

In [17]:
result = upload_preannotations(
    project_id=PROJECT_ID,
    annotation_format='coco_json',
    batch_annotation_dir=ANNOTATION_DIR
)

Batch Upload Started
Project ID: lynelle_additional_gibbon_85467
Annotation Format: coco_json
Total batch files: 1
Delay between uploads: 2s

[1/1] Uploading: annotations.json
  ✓ Success! Metadata: {'activity_id': '4d44bcc7-55bb-45ba-8e26-418f5dd5699e', 'files_not_updated': [], 'questions_ignored': []}

Batch Upload Complete!
Total batches: 1
Successful uploads: 1
Failed uploads: 0
Total time: 98.27s
